In [3]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('/Users/mhendriksen/Desktop/repositories/CLIP/src')

In [4]:
text_dict_path = '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/text_dict.pkl'
image_dict_path = '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/image_dict.pkl'

from utils import load_pkl

image_dict = load_pkl(image_dict_path)
text_dict = load_pkl(text_dict_path)

Loaded data from  /Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/image_dict.pkl
Loaded data from  /Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/text_dict.pkl


In [7]:
import yaml
import torch

############################
# load config files
############################
train_config_file = '/Users/mhendriksen/Desktop/repositories/CLIP/conf/train_conf.yaml'
dataset_config_file = '/Users/mhendriksen/Desktop/repositories/CLIP/conf/local_cub_conf.yaml'

with open(dataset_config_file) as file:
    config_ds = yaml.safe_load(file)
print(config_ds)

with open(train_config_file) as file:
    config_train = yaml.safe_load(file)
print(config_train)

config_train["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

{'CUB': {'dataset_root': '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/', 'csv_file': 'cub_1_cap_per_img.csv', 'image_folder': 'CUB_200_2011/images', 'columns_dtypes': {'image_file': 'str', 'class': 'str', 'eval_status': 'str', 'caption': 'str'}, 'content_type': {'image': 'image_file', 'text': 'caption'}, 'clip_version': 'clip-ViT-L-14', 'dict_file': 'test_dict.pkl'}}
{'DATASET': {'CUB': {'root': '/ivi/ilps/personal/mbiriuk/CUB_200_2011/', 'text_dict': 'text_dict.pkl', 'image_dict': 'image_dict.pkl', 'df_train': 'df_train.csv', 'df_test': 'df_test.csv', 'df_dev': 'df_dev.csv', 'model_path': '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/best_model_CUB_30E_128BS.pt'}, 'ABO': {'root': '/ivi/ilps/personal/mbiriuk/abo', 'text_dict': 'text_dict.pkl', 'image_dict': 'image_dict.pkl', 'df_train': 'df_train.csv', 'df_test': 'df_test.csv'}, 'fashion200k': {'root': '/ivi/ilps/personal/mbiriuk/fashion200k', 'text_dict': 'text_dict.pkl', 'image_dict': 'image_dict.pk

In [5]:
df_test_path = '/Users/mhendriksen/Desktop/repositories/CLIP/data/CUB/df_test.csv'
df_train_path = '/Users/mhendriksen/Desktop/repositories/CLIP/data/CUB/df_test.csv'

import pandas as pd

df_train = pd.read_csv(df_train_path, index_col=0)
df_test = pd.read_csv(df_test_path, index_col=0)

In [8]:
from dataset import ImageCaptionDataset, build_loader

train_loader = build_loader(
    dataset=ImageCaptionDataset,
    config=config_train,
    dataf=df_train,
    text_dict=text_dict,
    image_dict=image_dict,
    mode='train'
    )
valid_loader = build_loader(
    dataset=ImageCaptionDataset,
    config=config_train,
    dataf=df_test,
    text_dict=text_dict,
    image_dict=image_dict,
    mode='test'
    )


In [10]:
batch = next(iter(train_loader))

In [11]:
images, texts = batch[0], batch[1]

In [12]:
images.shape

torch.Size([128, 768])

In [27]:
import torch.nn as nn

input_dim = 768
num_heads = 8

self_attn = nn.MultiheadAttention(embed_dim=input_dim, num_heads=num_heads)

self_attn(images, images, images)



ValueError: not enough values to unpack (expected 3, got 2)

In [19]:
encoder_layer = nn.TransformerEncoderLayer(d_model=768, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
src = torch.rand(10, 5, 768)
out = transformer_encoder(src)

In [17]:
out

tensor([[[-0.2242, -1.0310, -0.2805,  ..., -0.8324, -1.2521,  0.1358],
         [-0.7744, -1.7791,  0.4502,  ...,  0.9238, -1.2825,  0.6134],
         [-0.6749, -0.2804,  0.1898,  ..., -0.3859, -0.5585,  0.9081],
         ...,
         [-0.3853, -1.8718,  0.6324,  ..., -0.2026, -0.3867,  0.3954],
         [ 0.6329, -1.3164,  0.1597,  ...,  0.5353, -1.1203, -0.1306],
         [-0.1416, -0.2096,  1.0230,  ..., -0.7071, -0.3981,  0.1989]],

        [[ 0.1905, -1.4942, -0.1423,  ..., -0.3503, -0.5814, -0.0370],
         [-0.2450, -1.6431,  0.1286,  ...,  0.2667, -1.4127,  0.7829],
         [-0.1701, -0.1983,  0.1089,  ..., -0.4841, -0.3509,  0.8635],
         ...,
         [-0.9135, -1.2016, -0.1343,  ...,  0.5964, -0.2756,  1.9324],
         [ 0.2297, -0.1577,  0.9443,  ...,  1.0557, -0.1343, -0.1161],
         [-1.2275, -0.9398,  0.4853,  ...,  0.6244, -0.6457,  0.3253]],

        [[ 0.1592, -1.6454,  0.6606,  ..., -0.3709, -0.4614, -0.3115],
         [ 0.3075, -1.2871,  0.3138,  ..., -0

In [32]:
class EncoderBlock(nn.Module):

    def __init__(self, input_dim, num_heads, dim_feedforward, dropout=0.0):
        """
        Inputs:
            input_dim - Dimensionality of the input
            num_heads - Number of heads to use in the attention block
            dim_feedforward - Dimensionality of the hidden layer in the MLP
            dropout - Dropout probability to use in the dropout layers
        """
        super().__init__()

        # Attention layer
        self.self_attn = nn.MultiheadAttention(input_dim, input_dim, num_heads)

        # Two-layer MLP
        self.linear_net = nn.Sequential(
            nn.Linear(input_dim, dim_feedforward),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(dim_feedforward, input_dim)
        )

        # Layers to apply in between the main layers
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Attention part
        attn_out = self.self_attn(x)
        x = x + self.dropout(attn_out)
        x = self.norm1(x)

        # MLP part
        linear_out = self.linear_net(x)
        x = x + self.dropout(linear_out)
        x = self.norm2(x)

        return x

In [33]:
block = EncoderBlock(input_dim=768, num_heads=8, dim_feedforward=2048)

block(images)

TypeError: forward() missing 2 required positional arguments: 'key' and 'value'